## Загрузим нужные библиотеки

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score

%matplotlib inline

In [2]:
df = pd.read_csv("https://vk.com/doc3811482_638349011?hash=sAyvkMFNDVnxNZUMeyP7GkdvJkMHMukqQuOkA2uZfkP&dl=k8J7fxKGaYY6V93yMCjmtwBBM3xcHSB2zxHXk4Ue7Oo")
df_test = pd.read_csv('https://vk.com/doc3811482_638349002?hash=CNhvkBU5xHnUMlsYHZy6DH6KoIZGQWdojFFkzCHxGGw&dl=rOYSmMLMTZhCe2tZ8lGocI2B7lMssZnBgO5qOunmKtH')

## Рассмотрим датасет по ближе

In [3]:
# df['m_avg_nontalk_duration'] = df.m_avg_duration - df.m_avg_talk_duration
# df_test['m_avg_nontalk_duration'] = df_test.m_avg_duration - df_test.m_avg_talk_duration
pd.set_option('display.max_columns', None)
df.spent_time_to_complete_hw.fillna(-2)
df


In [4]:
df.month_id = pd.to_datetime(df.month_id).values.astype(np.int64) // 10 ** 9
df_test.month_id = pd.to_datetime(df_test.month_id).values.astype(np.int64) // 10 ** 9
df.carts_created_at = pd.to_datetime(df.carts_created_at).values.astype(np.int64) // 10 ** 9
df_test.carts_created_at = pd.to_datetime(df_test.carts_created_at).values.astype(np.int64) // 10 ** 9
df['dlitelnost'] = df.month_id - df.carts_created_at
df_test['dlitelnost'] = df_test.month_id - df_test.carts_created_at
df_test.dlitelnost

In [5]:
df.ABC = df.ABC.astype('category')
df.ABC = df.ABC.cat.codes
df.ABC = df.ABC.astype('int')
df_test.ABC = df_test.ABC.astype('category')
df_test.ABC = df_test.ABC.cat.codes
df_test.ABC = df_test.ABC.astype('int')
df_test.ABC.value_counts()

# df = pd.concat([df, pd.get_dummies(df.ABC)])
# df_test = pd.concat([df_test, pd.get_dummies(df_test.ABC)])

# pd.set_option('display.max_columns', None)
# df.head(3)

In [6]:
df.ABC.value_counts()

In [7]:
df.communication_type = df.communication_type.astype('category')
df.communication_type = df.communication_type.cat.codes
df.communication_type = df.communication_type.astype('int')
df_test.communication_type = df_test.communication_type.astype('category')
df_test.communication_type = df_test.communication_type.cat.codes
df_test.communication_type = df_test.communication_type.astype('int')
df_test.communication_type.value_counts()

In [8]:
df.communication_type.value_counts()

In [9]:
df.promo = df.promo.apply(lambda x: 1 if x == '+' else 0)
df_test.promo = df_test.promo.apply(lambda x: 1 if x == '+' else 0)
df.promo

In [10]:
df.shape

In [11]:
df.info()

In [12]:
df.city= df.city.fillna(0)
df.city = df.city.apply(lambda x: 4 if x == 'Москва' else x)
df.city = df.city.apply(lambda x: 3 if x == 'Санкт-Петербург' else x)
df.city = df.city.apply(lambda x: 2 if x == 'Екатеринбург' else x)
df.city = df.city.apply(lambda x: 1 if x == 'Нижний Новгород' else x)
df.city = df.city.apply(lambda x: 5 if x != 0 and x != 1 and x != 2 and x != 3 and x != 4 else x)

df_test.city= df_test.city.fillna(0)
df_test.city = df_test.city.apply(lambda x: 4 if x == 'Москва' else x)
df_test.city = df_test.city.apply(lambda x: 3 if x == 'Санкт-Петербург' else x)
df_test.city = df_test.city.apply(lambda x: 2 if x == 'Екатеринбург' else x)
df_test.city = df_test.city.apply(lambda x: 1 if x == 'Нижний Новгород' else x)
df_test.city = df_test.city.apply(lambda x: 5 if x != 0 and x != 1 and x != 2 and x != 3 and x != 4 else x)

df.city

In [13]:
df.os = df.os.astype('category')
df.os = df.os.cat.codes
df.os = df.os.astype('int')
# df.os = df.os.fillna(-2)
df_test.os = df_test.os.astype('category')
df_test.os = df_test.os.cat.codes
df_test.os = df_test.os.astype('int')
# df_test.os = df_test.os.fillna(-2)
df_test.os.value_counts()

In [14]:
df.browser = df.browser.astype('category')
df.browser = df.browser.cat.codes
df.browser = df.browser.astype('int')
df_test.browser = df_test.browser.astype('category')
df_test.browser = df_test.browser.cat.codes
df_test.browser = df_test.browser.astype('int')
df_test.browser.value_counts()   

In [15]:
df.platform = df.platform.astype('category')
df.platform = df.platform.cat.codes
df.platform = df.platform.astype('int')
# df.platform = df.platform.fillna(3)
df_test.platform = df_test.platform.astype('category')
df_test.platform = df_test.platform.cat.codes
df_test.platform = df_test.platform.astype('int')
# df_test.platform = df_test.platform.fillna(3)
df.platform.value_counts()   

In [16]:
df.platform[191:211]

In [17]:
df.country.value_counts()

In [18]:
df.country = df.country.fillna(0) 
df.country = df.country.apply(lambda x: 2 if x == 'Россия' or x == 'РФ' else x)
df.country = df.country.apply(lambda x: 1 if x != 2 and x != 0 else x)
df_test.country = df_test.country.fillna(0) 
df_test.country = df_test.country.apply(lambda x: 2 if x == 'Россия' or x == 'РФ' else x)
df_test.country = df_test.country.apply(lambda x: 1 if x != 2 and x != 0 else x)

df.country.value_counts()

In [19]:
df = pd.concat([df,df_test])

cel = df.program_id.unique()[df.program_id.value_counts() > 40]  #при 40 самый лучший результат пока что
df.program_id = df.program_id.apply(lambda x: x if x in cel else 0)
df.program_id

df = pd.concat([df, pd.get_dummies(df.program_id, prefix = 'PR_id')], axis=1)
# df_test = pd.concat([df_test, pd.get_dummies(df_test.program_id, prefix='PR_id')], axis=1)
df_test = df[200000:]
df = df[0:200000]


Заменим все пропуски в данных

In [20]:
df.gender = df.gender.fillna(-1) 
sr = df.age_indicator.mean()
df.age_indicator = df.age_indicator.fillna(-1) 
df = df.fillna(0)
df_test = df_test.fillna(0)   #Здесь, чтобы не путать, можно пробовать -1 - не работает

Пострим на распределение данных

In [21]:
sns.countplot(x = "target" , data  = df).set_title('Распределение предсказываемой величены')

In [22]:
df["age_indicator"].hist(bins = 40).set_title('Распределение возраста')

Взглянем на несколько величин сразу

In [23]:
pd.set_option('display.max_columns', None)

df[5:10]



In [24]:
features = list(set(['age_indicator',
       'carts_created_at', 'spent_time_total', 'spent_time_to_complete_hw',
       'completed_hw', 'failed_hw', 'reworked_hw', 'interacted_hw',
       'avg_hw_mark', 'test_with_good_mark', 'test_with_great_mark',
       'webinars', 'avg_quiz_result', 'notes', 'hw_leader', 'lessons',
       'activity', 'bought_d1', 'bought_d2', 'bought_d3', 'bought_d4',
       'bought_d5', 'bought_avg_duration','dlitelnost','country']))

_ = df[features].hist(figsize=(20,12))

Обратим внимание на часть столбцов с постоянными значениями

In [25]:
df.spent_time_to_complete_hw.value_counts()

In [26]:
df.bought_d4.value_counts()

In [27]:
plt.rcParams['figure.figsize']=(15,15)

df_corr = df[["target",'age_indicator','month_id',
       'carts_created_at', 'spent_time_total',
       'completed_hw', 'failed_hw', 'reworked_hw', 'interacted_hw',
       'avg_hw_mark', 'test_with_good_mark', 'test_with_great_mark',
       'webinars', 'avg_quiz_result', 'promo', 'price',
       'communication_type', 'notes', 'hw_leader', 'lessons',
       'dlitelnost','city','os','browser']]

g = sns.heatmap(df_corr.corr(), square = True, annot=True)

## Выделим выборки

In [28]:
df.dtypes[df.dtypes == "object"].values
df_test.dtypes[df_test.dtypes == "object"].values

Поместим все столбцы object в массив, для их дальнейшего удаления

In [29]:
mass_object = df.dtypes[df.dtypes == "object"].index.values
mass_object_test = df_test.dtypes[df_test.dtypes == "object"].index.values

In [30]:
mass_object = np.append(mass_object, "target")
mass_object_test = np.append(mass_object_test, "target")


In [31]:
X = df.drop(mass_object, axis = 1)
# X = X.drop(['id','student_id'], axis = 1)
df_test = df_test.drop(mass_object_test, axis = 1)  
# df_test1 = df_test.drop(['id','student_id'], axis = 1)
y = df[["target"]]

In [32]:
X.shape


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)
print(X_train.shape)

## Обучение модели

### regr = RandomForestClassifier(n_jobs=-1, random_state=5)
#parametres = {'criterion': ['entropy'], 'n_estimators': [250], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [180]}   50 - 200
parametres = {'criterion': ['entropy'], 'n_estimators': [10], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [200]}  #идеально - 180 (на цель - 50)    
RegrCV = GridSearchCV(regr, parametres, cv = 5)
RegrCV.fit(X_train, y_train.values.ravel())
print(RegrCV.best_estimator_)
clf = RegrCV.best_estimator_
RegrCV.score(X_test, y_test.values.ravel())
#0.9601666666666666
#0.97165
#cel 50 - лучше 20 и 100. Где-то здесь максимум.
#0.972 на cel 40

#0.96305 - c програм 0.9585 - без програм 0.96255 - c новой темой #0.96

In [43]:
regr = RandomForestClassifier(n_jobs=-1, random_state=5)
#parametres = {'criterion': ['entropy'], 'n_estimators': [250], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [180]} 50 - 200 
parametres = {'criterion': ['entropy'], 'n_estimators': [500], 'max_depth': [35], 'min_samples_leaf': [1], 'max_features': [200]} #идеально - 180 (на цель - 50)
RegrCV = GridSearchCV(regr, parametres, cv = 5) 
RegrCV.fit(X_train, y_train.values.ravel()) 
print(RegrCV.best_estimator_) 
clf = RegrCV.best_estimator_ 
RegrCV.score(X_test, y_test.values.ravel()) 
#0.9601666666666666 #0.97165 
#cel 50 - лучше 20 и 100. Где-то здесь максимум. 
#0.972 на cel 40

#0.96305 - c програм 0.9585 - без програм 0.96255 - c новой темой #0.96

In [44]:
clf.feature_importances_

features = X.columns
importances = clf.feature_importances_[0:20]
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

## Оценка точности

In [45]:
pred = clf.predict(X_test)

In [46]:
y_test.head(5)

In [47]:
print('Recall: ', recall_score(y_test, pred, average='macro'))
print('Precision: ', precision_score(y_test, pred, average='macro'))
0.2* recall_score(y_test, pred, average='macro') + 0.8* precision_score(y_test, pred, average='macro')
#0.8460682083585125 - лучше, когда ABC одним числом
# 0.834670322089657 - без студентов и id
# 0.8432280950272583 - c длительностью
# 0.8731528445452765 - c улучшенными city и со студентами.0.8803246702006294 - c os и browser
# 0.8803246702006294 - c os и browser
# 0.8790399051646276 - дополнительно с платформой
# 0.8882240582947984 - страны и улучшенные сити, все еще с платформой. И новое дерево.
# 0.897066900229388 - добавил program_id как фичи
# 0.9184587043988088
#0.920 - при тесте на 0.05 тестовой выборки
# 0.920917427869218 - на 40-ка
#0.920917427869218


In [48]:
predproba = clf.predict_proba(X_test)
predproba

# Прогноз

In [49]:
df_test

In [50]:
PredT= clf.predict(df_test)
PredT = pd.DataFrame(PredT, columns=['target'])
PredT['id'] = df_test.id
neworder = ['id','target']
PredT=PredT.reindex(columns=neworder)
PredT


In [51]:
PredT.target

In [52]:
output3 = PredT #верные названия для сдачи работы
# output3.insert('id,target', predT.index)
#output3 = output3.set_index('document_id')
output3.to_csv('subnetology.csv', index=False) #, index=False
output3.head(5)

# Градиентный Бустинг

In [ ]:
stop
import lightgbm
from sklearn.metrics import r2_score

def lgbm_r2(y_true, y_pred):
    return 'r2', r2_score(y_true, y_pred), True

lgbm_views = lightgbm.LGBMClassifier(num_leaves=5, n_estimators=60000, learning_rate=0.05, depth=5)
lgbm_views = lgbm_views.fit(X_train, y_train,
                            eval_set=[(X_train, y_train), (X_test, y_test)],
                            eval_names=['train', 'val'], eval_metric='logloss', callbacks=[lightgbm.log_evaluation(500)])


In [ ]:
lgbm_views.score(X_test, y_test)

In [ ]:
lightgbm.plot_metric(lgbm_views, figsize=(15, 10));

In [ ]:
predlgb = lgbm_views.predict(X_test)
print('Recall: ', recall_score(y_test, predlgb, average='macro'))
print('Precision: ', precision_score(y_test, predlgb, average='macro'))
0.2* recall_score(y_test, predlgb, average='macro') + 0.8* precision_score(y_test, predlgb, average='macro')

# Нейросети

In [ ]:

import torch
from sklearn.preprocessing import StandardScaler

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [ ]:
X_train, X_test, y_train, y_test

ss = StandardScaler().fit(X_train)
X_train = ss.transform(X_train)
X_val = ss.transform(X_test)
X_test1 = ss.transform(df_test)

In [ ]:
device = get_device()

X_t = torch.from_numpy(X_train).float().to(device)
y_t = df_to_tensor(y_train).long()
X_v = torch.from_numpy(X_val).float().to(device)
y_v = df_to_tensor(y_test).long()
X_testtenz = torch.from_numpy(X_test1).float().to(device)

X_t

In [ ]:
len(X_v)

In [ ]:
class DetailNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(DetailNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(X_train.shape[1], 256)
        self.activ1 = torch.nn.ReLU()
        self.bn1 = torch.nn.BatchNorm1d(num_features=256)
        self.dout1 = torch.nn.Dropout(p=0.3)
        self.fc5 = torch.nn.Linear(256, 256)
        self.activ4 = torch.nn.ReLU()
        self.bn4 = torch.nn.BatchNorm1d(num_features=256)
        self.fc4 = torch.nn.Linear(256, 64)
        self.activ3 = torch.nn.ReLU()
        self.bn3 = torch.nn.BatchNorm1d(num_features=64)
        self.fc2 = torch.nn.Linear(64, 16)
        self.activ2 = torch.nn.ReLU()
#         self.bn2 = torch.nn.BatchNorm1d(num_features=n_hidden_neurons)
        self.fc3 = torch.nn.Linear(16, 5)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.bn1(x)
        x = self.dout1(x)
        x = self.fc5(x)
        x = self.activ4(x)
        x = self.bn4(x)
        x = self.fc4(x)
        x = self.activ3(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.activ2(x)
#         x = self.bn2(x)
        x = self.fc3(x)
        x = self.sm(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
detail_net = DetailNet(60).to(device)


In [ ]:
def loss(pred, target):
    squares = (pred - target) ** 2
    return squares.mean()

optimizer = torch.optim.Adam(detail_net.parameters(), weight_decay=0.00005, 
                             lr=0.001)

In [ ]:
batch_size = 1500
detail_net = detail_net.train()
for epoch in range(500):
    order = np.random.permutation(len(X_t))
    for start_index in range(0, len(X_t), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_t[batch_indexes].to(device)
        y_batch = y_t[batch_indexes].to(device)
        
        preds = detail_net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 50 == 0:
        test_preds = detail_net.forward(X_v)
        print(((test_preds - y_v)*(test_preds - y_v)).float().mean())
        
detail_net = detail_net.eval()

In [ ]:
preds = detail_net.forward(X_v)
preds

In [ ]:
print('Recall: ', recall_score(y_v, preds, average='macro'))
print('Precision: ', precision_score(y_v, preds, average='macro'))
0.2* recall_score(y_v, preds, average='macro') + 0.8* precision_score(y_v, preds, average='macro')